# Determining the Ising model numerically
For a good review of what this is all about, see [Inverse statistical problems: from the inverse Ising problem to data science](https://arxiv.org/abs/1702.01522). We want to solve for the variables $\boldsymbol{h}\in\mathbb{R}^{N}$ and $\boldsymbol{J}\in\mathbb{R}^{N\times(N-1)/2}$ such that our Ising model reproduces observed averages $\langle \sigma_i \rangle^D$ and correlations $\langle \sigma_i \sigma_j \rangle ^D$. We start by taking a naïve approach, using gradient ascent on the log-likelihood function:
$$
\begin{aligned}
L_{D}(\boldsymbol{h}, \boldsymbol{J}) &=\frac{1}{M} \ln p(\mathrm{D} \mid \boldsymbol{J}, \boldsymbol{h}) \\
&=\sum_{i<j} J_{i j}\left\langle\sigma_{i} \sigma_{j}\right\rangle^{D}+\sum_{i} h_{i}\left\langle\sigma_{i}\right\rangle^{D}-\ln Z(\boldsymbol{h}, \boldsymbol{J})
\end{aligned}
$$
The partial derivatives of the log-likelihood function are:
$$
\begin{aligned}
\frac{\partial L_{D}}{\partial h_i} =& \langle \sigma_i \rangle^D - \langle \sigma_i \rangle \\
\frac{\partial L_{D}}{\partial J_{ij}} =& \langle \sigma_i \sigma_j  \rangle^D - \langle \sigma_i \sigma_j \rangle
\end{aligned}
$$
and the gradient ascent update rule is:
$$
\begin{aligned}
h_i^{(t+1)} =& h_i^{(t)} + \lambda \frac{\partial L_{D}}{\partial h_i} \\
J_{ij}^{(t+1)} =& J_{ij}^{(t)} + \lambda \frac{\partial L_{D}}{\partial J_{ij}} 
\end{aligned}
$$
where $\lambda$ is the learning rate.

In [1]:
import numpy as np

In [30]:
class Ising:
    """
    Represents an Ising model.
    Variables:
        N - no. spins
        av_s - vector of expectations for each spin
        av_ss - matrix of pairwise correlations
        lr - learning rate
        spin_vals - the values each spin takes on, typically [0,1] or [-1,1]
        states - matrix of all possible states
        h - vector of the local magnetic fields 
        J - matrix of the pairwise couplings 
        Z - the current value of the partition function
    """
    def __init__(self, N, avgs, corrs, lr=0.1, spin_vals=[0,1]):
        # set user input
        self.N = N
        self.avgs = avgs
        self.corrs = corrs
        self.lr = lr
        self.spin_vals = spin_vals
        # determine all states
        self.states = np.array([self.to_binary(n) for n in range(2**N)]) 
        # randomly initialise h and J
        self.h = np.random.random_sample((N))
        self.J = np.triu( np.random.random_sample((N,N)), 1)
        # work out the partition function Z
        self.Z = self.calc_Z()
    
    # Methods for calculating probabilities and expectations
    def expectation(self, f, ind, p):
        """
        Returns the sum over all states of the function f, weighted by p. 
        Args:
            f - a function of a subset of the spins
            ind - indices of the spins which are involved in f
            p - a function of the state, for instance the probability p of observing the state
        """
        exp = 0
        for s in self.states:
            exp += f( [s[i] for i in ind] ) * p(s)
        
        return exp

    def p(self, s):
        """
        Returns the normalized probability of the state s given the model parameters 
        Args:
            s - np.array of the state, e.g. np.array([0,0,1]), here the third neuron fires
        """
        return np.exp(-self.H(s)) / self.Z

    def p_unnormalized(self, s):
        """
        Returns the unnormalized probability (not divided Z) of the state s given the model parameters 
        Args:
            s - np.array of the state
        """
        return np.exp(-self.H(s))

    def H(self, s):
        """
        Return the hamiltonian H(s) of the state s
        Args:
            s - np.array of the state
        """
        return -self.h.dot(s) - s@self.J@s 
            
    def calc_Z(self):
        """ 
        Calculates the partition function Z based on the current h and J.
        """
        # (the lambda function just returns 1 since this is just a sum of p over all states) 
        Z = self.expectation(lambda args: 1, [], self.p_unnormalized) 
        return Z 
    
    def to_binary(self, n):
        """
        Returns a binary rep of the int n as an array of size N, e.g. Assuming N = 5, 3 -> np.array([0,0,0,1,1]) 
        """
        b = np.zeros(self.N)
        for i in range(N):
            if n % 2 == 1: b[N-1-i]=1 # index N-1-i otherwise numbers are reversed
            n//=2
            if n==0: break
        return b

    # Methods for gradient ascent
    def gradient_ascent(self):
        """
        Performs gradient ascent on the log-likelihood and updates h and J
        """
        steps = 100
        for n in range(steps): #update this condition to check accuracy
            # work out corrections to h
            h_new = self.h
            for i in range(self.N):
                pass

            # work out corrections to J
            J_new = self.J
            for i in range(self.N-1):
                for j in range(i+1,N):
                    pass

            # perform the update
            self.h = h_new
            self.J = J_new
    

IndentationError: expected an indented block (<ipython-input-30-be85bb733a04>, line 98)

In [35]:
for i in range(3):
    for j in range(i+1,4):
        print(i,j,end=", ")
    print()

0 1, 0 2, 0 3, 
1 2, 1 3, 
2 3, 


## Create and train an Ising model on some fictional data

In [21]:
N = 4
avgs = 0.5*np.ones(N) # prob of every neuron firing in a window is 0.5
corrs = 0.2*np.triu(np.ones((N,N)),1) # prob of 2 neurons firing in the same window is 0.2 
print(avgs,corrs, sep="\n")

[0.5 0.5 0.5 0.5]
[[0.  0.2 0.2 0.2]
 [0.  0.  0.2 0.2]
 [0.  0.  0.  0.2]
 [0.  0.  0.  0. ]]


In [23]:
ising = Ising(N, avgs, corrs)
ising.__dict__ # all the class variables

{'N': 4,
 'avgs': array([0.5, 0.5, 0.5, 0.5]),
 'corrs': array([[0. , 0.2, 0.2, 0.2],
        [0. , 0. , 0.2, 0.2],
        [0. , 0. , 0. , 0.2],
        [0. , 0. , 0. , 0. ]]),
 'lr': 0.1,
 'spin_vals': [0, 1],
 'states': array([[0., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 1., 1., 0.],
        [0., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 1.],
        [1., 0., 1., 0.],
        [1., 0., 1., 1.],
        [1., 1., 0., 0.],
        [1., 1., 0., 1.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]]),
 'h': array([0.14756489, 0.92124773, 0.04867979, 0.71479699]),
 'J': array([[0.        , 0.23619783, 0.61125111, 0.92269517],
        [0.        , 0.        , 0.44777157, 0.27042526],
        [0.        , 0.        , 0.        , 0.04394949],
        [0.        , 0.        , 0.        , 0.        ]]),
 'Z': 170.98697504882213}

In [25]:
ising.p(np.ones(N)) #probability of all neurons firing

0.4597798200658618